# Ising Model in 1D and 2D: Metropolis Monte Carlo Simulations

In this 

In [ ]:
#the first time you run this, you may need to install these packages

In [ ]:
!pip install pillow h5py cython --user

In [ ]:
import ipywidgets as widgets
import numpy as np
from ipywidgets import interact
from PIL import Image
from numpy.fft import rfft2, irfft2, fftshift
from numpy.fft import rfft, irfft
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import IPython.display as display
import h5py

In [ ]:
%load_ext cython
%matplotlib notebook

def autocorrelation_1D(img):
    fft_img = rfft(img)
    xs = np.arange(len(img)) - 0.5 * len(img)
    return xs, fftshift(irfft(fft_img * np.conj(fft_img)))/np.prod(img.shape)

def display_ising_sequence_1D(images):
    def _show(frame=(0, len(images) - 1)):
        return display_spin_field(images[frame])
    return interact(_show)

def random_spin_field_1D(N):
    return np.random.choice([-1, 1], size=(N))

def magnetization(img):
    return np.sum(img)/np.prod(img.shape)

def display_spin_field_1D(field):
    img = np.array([field]* 40)
    img = Image.fromarray(np.uint8((img + 1) * 0.4 * 255 + 25.5))  # 0 ... 255
    img.save('temp_1d.png')
    with open('temp_1d.png', 'rb') as input_file:
        img = input_file.read()
    return widgets.Image(value=img, layout=widgets.Layout(width='90%'))

def autocorrelation(img):
    fft_final = rfft2(img)
    return fftshift(irfft2(fft_final * np.conj(fft_final)))
    
def radial_autocorrelation(img):
    ac = autocorrelation(img)
    shape = img.shape
    max_r = int(np.min(shape) / np.sqrt(2)) + 1
    hist = np.zeros(max_r + 1)
    counts = np.zeros(max_r + 1)
    for i in range(shape[0]):
        for j in range(shape[1]):
            r = int(np.sqrt((i - 0.5 * shape[0]) ** 2 + (j - 0.5 * shape[1])**2))
            hist[r] += ac[i, j]
            counts[r] += 1
            
    rhist = hist[counts > 0]/counts[counts > 0] / (np.prod(img.shape))
    rs = np.arange(0, max_r)
    return rs, rhist

def display_ising_sequence(images, sweeps, eng, mag, ac_ax, mag_ax, e_ax):
    def _show(sweep):
        ac_ax.clear()
        rs, ac = radial_autocorrelation(images[sweep])
        
        ac_ax.plot(rs, ac)
        ac_ax.set_title('Radial Autocorrelation')
        ac_ax.set_xlabel('$\Delta r$')
        ac_ax.set_ylabel('Autocorrelation')
        ac_ax.set_ylim(-0.5, 1.2)
        
        e_ax.clear()
        e_ax.plot(sweeps, eng, '-k')
        e_ax.plot([sweeps[sweep]], [eng[sweep]], 'or')
        e_ax.set_title('Average Energy  = {:0.2f}'.format(eng[sweep]))
        e_ax.set_xlabel('MC sweeps')
        e_ax.set_ylabel('Energy')
        e_ax.set_ylim(-8, 0)
        
        mag_ax.clear()
        mag_ax.plot(sweeps, mag, '-k')
        mag_ax.plot([sweeps[sweep]], [mag[sweep]], 'or')
        mag_ax.set_title('Magnetization = {:0.2f}'.format(mag[sweep]))
        mag_ax.set_xlabel('MC sweeps')
        mag_ax.set_ylabel('Magentization')
        mag_ax.set_ylim(-1, 1)

        return display_spin_field(images[sweep])
    
    sweep_selector = widgets.SelectionSlider(
        options=[('{:d}'.format(sweep), s) for s, sweep in enumerate(sorted(sweeps))],
        description='sweep = ',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        layout=widgets.Layout(width='80%')
    )
    return interact(_show, sweep=sweep_selector)

def display_ising_sequence_1D(images, sweeps, eng, mag, ac_ax, mag_ax, e_ax):
    def _show(sweep):
        print(sweep)
        ac_ax.clear()
        xs, ac = autocorrelation_1D(images[sweep])
        
        ac_ax.plot(xs, ac)
        ac_ax.set_title('1-D Autocorrelation')
        ac_ax.set_xlabel('$\Delta x$')
        ac_ax.set_ylabel('Autocorrelation')
        ac_ax.set_ylim(-0.5, 1.2)
        
        e_ax.clear()
        e_ax.plot(sweeps, eng, '-k')
        e_ax.plot([sweeps[sweep]], [eng[sweep]], 'or')
        e_ax.set_title('Average Energy  = {:0.2f}'.format(eng[sweep]))
        e_ax.set_xlabel('MC sweeps')
        e_ax.set_ylabel('Energy')
        e_ax.set_ylim(-2, 0)
        
        mag_ax.clear()
        mag_ax.plot(sweeps, mag, '-k')
        mag_ax.plot([sweeps[sweep]], [mag[sweep]], 'or')
        mag_ax.set_title('Magnetization = {:0.2f}'.format(mag[sweep]))
        mag_ax.set_xlabel('MC sweeps')
        mag_ax.set_ylabel('Magentization')
        mag_ax.set_ylim(-1, 1)
        
        return display_spin_field_1D(images[sweep])
    
    sweep_selector = widgets.SelectionSlider(
        options=[('{:d}'.format(sweep), s) for s, sweep in enumerate(sorted(sweeps))],
        description='sweep = ',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        layout=widgets.Layout(width='80%')
    )
    return interact(_show, sweep=sweep_selector)

def random_spin_field(N, M):
    return np.random.choice([-1, 1], size=(N, M))

def display_spin_field(field):
    img = Image.fromarray(np.uint8((field + 1) * 0.4 * 255 + 25.25))
    img.save('temp_2d.png')
    with open('temp_2d.png', 'rb') as input_file:
        img = input_file.read()
    return widgets.Image(value=img, layout=widgets.Layout(width='60%'))


In [ ]:
%%cython

cimport cython

import numpy as np
cimport numpy as np

from libc.math cimport exp
from libc.stdlib cimport rand
cdef extern from "limits.h":
    int RAND_MAX


@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_step_1D(np.int64_t[:] field, float beta=0.4, int n_moves=1):
    cdef int N = field.shape[0]
    cdef int n_offset, n
    for i in range(n_moves):
        n_offset = np.random.choice(np.arange(2))
        for n in range(n_offset, N, 2):
            _cy_ising_update_1D(field, n, beta)
    return np.array(field)


@cython.boundscheck(False)
@cython.wraparound(False)
cdef _cy_ising_update_1D(np.int64_t[:] field, int n, float beta):
    cdef int total = 0
    cdef int N = field.shape[0]
    cdef int i
    for i in range(n-1, n+2):
        if i == n:
            continue
        total += field[i % N]
    cdef float dE = 2 * field[n] * total
    
    if dE <= 0:
        field[n] *= -1
    elif exp(-dE * beta) * RAND_MAX > rand():
        field[n] *= -1



@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_step(np.int64_t[:, :] field, float beta=0.4, int n_moves=1):
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int n_offset, m_offset, n, m
    n_offsets = np.random.choice(range(2), n_moves)
    m_offsets = np.random.choice(range(2), n_moves)
    for (n_offset, m_offset) in zip(n_offsets, m_offsets):
        for n in range(n_offset, N, 2):
            for m in range(m_offset, M, 2):
                _cy_ising_update(field, n, m, beta)
    return np.array(field)


@cython.boundscheck(False)
@cython.wraparound(False)
cdef _cy_ising_update(np.int64_t[:, :] field, int n, int m, float beta):
    cdef int total = 0
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int i, j
    for i in range(n-1, n+2):
        for j in range(m-1, m+2):
            if i == n and j == m:
                continue
            total += field[i % N, j % M]
    cdef float dE = 2 * field[n, m] * total
    if dE <= 0:
        field[n, m] *= -1
    elif exp(-dE * beta) * RAND_MAX > rand():
        field[n, m] *= -1
        
@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_energy(np.int64_t[:, :] field):
    cdef int total = 0
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int i, j
    cdef int n, m
    cdef float total_E = 0
    cdef float row_E = 0 
    for n in range(N):
        row_E = 0 
        for m in range(M):
            total = 0
            for i in range(n-1, n+2):
                for j in range(m-1, m+2):
                    if i == n and j == m:
                        continue
                    total += field[i % N, j % M]
            row_E += - field[n, m] * total
        total_E += row_E
    return total_E /(N * M)


@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_energy_1D(np.int64_t[:] field):
    cdef int total = 0
    cdef int N = field.shape[0]
    cdef int i
    cdef int n
    cdef float total_E = 0
    for n in range(N):
        total = 0
        for i in range(n-1, n+2):
            if i == n:
                continue
            total += field[i % N]
        total_E += - total * field[n]
    return total_E /(N)


In [ ]:
def ising_1d_simulation(beta, number_of_samples = 100, moves_per_sweep = 1000):
    images = []
    step = random_spin_field_1D(500)
    images.append(step)

    sweep_number = np.arange(0, (number_of_samples + 1) * moves_per_sweep, moves_per_sweep)
    magnet = np.empty(number_of_samples + 1)
    energy = np.empty(number_of_samples + 1)
    magnet[0] = magnetization(step)
    energy[0] = cy_ising_energy_1D(step)

    progress = widgets.FloatProgress(
        value=1,
        min=0,
        max=number_of_samples + 1,
        step=0.1,
        description='Simulating!',
        bar_style='info',
        orientation='horizontal',
        layout=widgets.Layout(width='90%')
    )
    display.display(progress)
    for sweep in range(1, number_of_samples + 1):
        progress.value = sweep
        step = cy_ising_step_1D(step.copy(), beta=beta, n_moves=moves_per_sweep)
        images.append(step)
        energy[sweep] = cy_ising_energy_1D(step)
        magnet[sweep] = magnetization(step)
    
    display.clear_output()
    spec = GridSpec(ncols=12, nrows=12)
    fig = plt.figure(figsize=(9, 9))
    ax1 = fig.add_subplot(spec[0:7,:])
    ax2 = fig.add_subplot(spec[9:, 0:5])
    ax3 = fig.add_subplot(spec[9:,7:])

    
    display_ising_sequence_1D(images, sweep_number, energy, magnet, ax1, ax2, ax3);
    return dict(
        images=images, 
        number_of_samples=number_of_samples,
        moves_per_sweep=moves_per_sweep,
        beta=beta,
    )

def ising_2d_simulation(beta, number_of_samples = 100, moves_per_sweep = 1000, size=200):
    images = []
    step = random_spin_field(size, size)
    images.append(step)

    sweep_number = np.arange(0, (number_of_samples + 1) * moves_per_sweep, moves_per_sweep)
    magnet = np.empty(number_of_samples + 1)
    energy = np.empty(number_of_samples + 1)
    magnet[0] = magnetization(step)
    energy[0] = cy_ising_energy(step)
    
    progress = widgets.FloatProgress(
        value=1,
        min=0,
        max=number_of_samples + 1,
        step=0.1,
        description='Simulating!',
        bar_style='info',
        orientation='horizontal',
        layout=widgets.Layout(width='90%')
    )
    display.display(progress)
    for sweep in range(1, number_of_samples + 1):
        progress.value = sweep
        step = cy_ising_step(step.copy(), beta=beta, n_moves=moves_per_sweep)
        images.append(step)
        energy[sweep] = cy_ising_energy(step)
        magnet[sweep] = magnetization(step)

    display.clear_output()
    spec = GridSpec(ncols=12, nrows=12)
    fig = plt.figure(figsize=(9, 9))
    ax1 = fig.add_subplot(spec[0:7,:])
    ax2 = fig.add_subplot(spec[9:, 0:5])
    ax3 = fig.add_subplot(spec[9:,7:])

    
    display_ising_sequence(images, sweep_number, energy, magnet, ax1, ax2, ax3);
    return dict(
        images=images, 
        number_of_samples=number_of_samples,
        moves_per_sweep=moves_per_sweep,
        beta=beta,
    )

def save_data(data, filename):
    with h5py.File('simulations/' + filename, 'w') as outf:
        outf.attrs.moves_per_sweep = data['moves_per_sweep']
        outf.attrs.number_of_samples = data['number_of_samples']
        outf.images = data['images']
        outf.attrs.beta = data['beta']

In [ ]:
filename = '1D_trial_3.hdf5'
data = ising_1d_simulation(beta=4)
save_data(data, filename=filename)

In [ ]:
# This will take ~60 seconds to complete.
# A plot will appear when it's done
filename = '2D_trial_3.hdf5'
data = ising_2d_simulation(beta=0.2, moves_per_sweep=10, size=200)
save_data(data, filename=filename)

When you run the command below, you should see a list of all of your files. There should be twelve of them!

In [ ]:
!ls simulations